In [4]:
from xltpl.writerx import BookWriter  # writerx = para .xlsx (openpyxl)
import sys
import mysql.connector
import xlsxwriter
import pandas as pd
import sys
import mysql.connector
import os
from dotenv import load_dotenv
import num2words
import numpy as np
load_dotenv()
#ESTE ARGUMENTO NO SE USA EN ESTE REPORTE, SERÁ 0 SIEMPRE UWU
#configurar la conexion a la base de datos
DB_USERNAME = os.getenv('DB_USERNAME')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')

a_color='#354F84'
b_color='#91959E'
# Conectar a DB
cnx = mysql.connector.connect(user=DB_USERNAME,
                              password=DB_PASSWORD,
                              host='localhost',
                              port=DB_PORT,
                              database=DB_DATABASE,
                              use_pure=False)
#Seccion para traer informacion de la base

# join para cobros
# cobros=pd.read_sql('Select cobros.* ,customers.customer,internal_orders.invoice, users.name from ((cobros inner join internal_orders on internal_orders.id = cobros.order_id) inner join customers on customers.id = internal_orders.customer_id )inner join users on cobros.capturo=users.id',cnx)

# Carga la plantilla

# id=str(sys.argv[1])
# id=706#6
#id=754#5
#id=131#4 items
#id=679#3
id=413#1
#traer datos de los pedidos
order=pd.read_sql(f"""select internal_orders.* ,customers.clave,customers.alias,
coins.exchange_sell, coins.coin, coins.symbol,coins.code
from ((
    internal_orders
    inner join customers on customers.id = internal_orders.customer_id )
    inner join coins on internal_orders.coin_id = coins.id)
                    where internal_orders.id  = {id}
     """,cnx)
order['status']=order['status'].str.upper()
for col in ["reg_date","date_delivery","instalation_date"]:
    order[col] = pd.to_datetime(order[col], format="%Y-%m-%d").dt.strftime("%d-%m-%Y")
customer=pd.read_sql(f"select * from customers where id ={order['customer_id'].values[0]}",cnx)

marca=pd.read_sql(f"select * from marcas where id ={order['marca'].values[0]}",cnx)
if(len(marca)>0):
    marca=marca['name'].values[0]
else:
    marca='Sin marca seleccionada'
seller=pd.read_sql(f"select * from sellers where id ={order['seller_id'].values[0]}",cnx)
customer_adress=pd.read_sql(f"select * from 	customer_shipping_addresses	 where customer_id ={order['customer_id'].values[0]}",cnx)
contacts=pd.read_sql(f"select customer_contacts.* from order_contacts  inner join customer_contacts on customer_contacts.id=order_contacts.contact_id where order_contacts.order_id ={order['id'].values[0]}",cnx)
coin=pd.read_sql(f"select * from coins where id ={order['coin_id'].values[0]}",cnx)
items=pd.read_sql(f"select * from items where internal_order_id ={order['id'].values[0]}",cnx)
required_signatures=pd.read_sql(f"""select signatures.*,authorizations.titulo 
                                from authorizations  inner join signatures 
                                on authorizations.id=signatures.auth_id
                                 where signatures.order_id ={order['id'].values[0]}""",cnx)
for i in range(5-len(required_signatures)):
    required_signatures=pd.concat([required_signatures,pd.DataFrame({'firma': ' ','titulo': 'FIRMA','status':0},index=[len(required_signatures)+i])])
pagos=pd.read_sql(f"select * from payments where order_id ={order['id'].values[0]}",cnx)

pagos['dia_anio']=pd.to_datetime(pagos["date"], format="%Y-%m-%d").dt.dayofyear
pagos['semana']=pd.to_datetime(pagos["date"], format="%Y-%m-%d").dt.isocalendar().week
pagos['date'] = pd.to_datetime(pagos['date'], format="%Y-%m-%d").dt.strftime("%d-%m-%Y")
comisiones=pd.read_sql(f"""select comissions.*,sellers.iniciales,sellers.seller_name from comissions
                       inner join sellers on sellers.id=comissions.seller_id
                        where order_id ={order['id'].values[0]}""",cnx)
letter_total=num2words.num2words(order['total'].values[0], lang='es').upper()

#variables para contrlolar la longitud de la pagina
len_page=58
npages=3


# # Datos a renderizar (puedes anidar dicts/listas sin problema)
#en la primera hoja se usan 37 celdas fuera de los ciclos
first_page_cells=37

if(len(items)>3):
    len_tabla_inf=8
    item_pre_completer=np.arange(0,len_tabla_inf+1)
    item_completer=np.arange(0,(len_page*2)-(6*len(items)+first_page_cells)-len_tabla_inf)
    npages=npages+1
else:
    item_pre_completer=[]
    item_completer=np.arange(0,len_page-(6*(3)+first_page_cells)+1)
    print(len(item_completer),'item completer')
if(len(items)>4):
    item_pre_completer=[]
    
    item_completer=np.arange(0,(len_page*2)-(6*len(items)+first_page_cells)-2)
if(len(items)>12):
    item_completer=[]

second_page_cells=37
if(len(pagos)>20):
    completer=np.arange(0,(len_page*2)-(len(pagos)+second_page_cells))
    npages=npages+1
else:
    completer=np.arange(0,len_page-(len(pagos)+second_page_cells))
if(len(pagos)>75):
    completer=[]

print(len(item_completer),'item completer')
print(len(item_pre_completer),'item pre completer')
com_completer=np.arange(0,len_page-(len(comisiones)+24))


payload = {
    "fecha": "2025-08-16",
    'letter_total':letter_total,
    'completer':completer,
    'item_pre_completer':item_pre_completer,
    'item_completer':item_completer,
    'marca':marca,
    'com_completer':com_completer,
    "palomita": "check.png",
}

for df,name in zip([order,customer,seller,customer_adress,coin],["order","customer","seller","customer_adress","coin"]):
    aux_dict={}
    print(name)
    for col in df.columns:
        aux_dict.update({col : df[col].values[0]})
    payload.update({name:aux_dict})
#Colecciones de datos con mas de una fila en el dataframe
for df,name in zip([items,required_signatures,contacts,pagos,comisiones],["items","signatures","contacts","pagos","comisiones"]):
    objects=[]
    print(name,len(df))
    aux_dict={}
    for i in range(len(df)):
        item={}
        for col in df.columns:
            item.update({col : df[col].values[i]})
        objects.append(item)
    if(name=='signatures'):
        for i in range(5-len(required_signatures)):
            objects.append({'firma':'FIRMA','titulo':'Autorizacion'})
    if((name=='contacts')&(len(df)<3)):
        for i in range(3-len(df)):
            objects.append({'customer_contact_name':'NA','customer_contact_office_phone':'-','customer_contact_office_phone_ext':'-',
                            'customer_contact_mobile':'-','customer_contact_email':'-'}) 
    if((name=='items')&(len(df)<3)):
        for i in range(3-len(items)):
            objects.append({'amount':'0',})         
    payload.update({name:objects})
# #Renderizar excel
# if(len(items)<=3):
#     writer = BookWriter('plantilla_pedido_confidential.xlsx')
# else:
#     writer = BookWriter('plantilla_pedido_confidential_large.xlsx')
writer = BookWriter('plantilla_pedido_confidential.xlsx')
# Renderiza (se pasa una lista de payloads si quieres varias “páginas/hojas”)
writer.render_book([payload])

# Guarda el resultado
writer.save(f'storage/report/impresion_pedido_confidential{id}.xlsx')


#configurar n pages
from openpyxl import load_workbook

# Abrir el archivo generado
wb = load_workbook(f'storage/report/impresion_pedido_confidential{id}.xlsx')
ws = wb.active  # o el nombre de tu hoja ws = wb["Hoja1"]

# Ajustar impresión
ws.page_setup.fitToWidth = 1     # 1 página de ancho
ws.page_setup.fitToHeight = npages    # m páginas de alto 

# También puedes asegurar que el escalado automático esté activado
# ws.page_setup.scale = None  

# Guardar el archivo modificado
wb.save(f'storage/report/impresion_pedido_confidential{id}.xlsx')

c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site

35 item completer
0 item pre completer
order
customer
seller
customer_adress
coin
items 7
signatures 5
contacts 0
pagos 4
comisiones 4
<openpyxl.drawing.image.Image object at 0x0000022044DDD870> not Img
<openpyxl.drawing.image.Image object at 0x0000022044DDD810> not Img
<openpyxl.drawing.image.Image object at 0x0000022044DDD8D0> not Img
<openpyxl.drawing.image.Image object at 0x0000022044DDD960> not Img
<openpyxl.drawing.image.Image object at 0x0000022044DDD900> not Img
<openpyxl.drawing.image.Image object at 0x0000022044DDD930> not Img


In [5]:
letter_total.upper()

'NOVECIENTOS NOVENTA Y CINCO MIL CUARENTA Y CUATRO PUNTO CINCO DOS'

In [2]:
for i in range(7):
    print(i*58)

0
58
116
174
232
290
348


In [45]:
npages

5

In [55]:
required_signatures

,id,order_id,auth_id,status,created_at,updated_at,firma,titulo
0,2006.0,815.0,2.0,0,2025-08-29,2025-08-29,None,GERENTE DE VENTAS
1,2007.0,815.0,3.0,0,2025-08-29,2025-08-29,None,GERENTE ADMINISTRATIVO
2,NaN,NaN,NaN,0,NaN,NaN,,FIRMA
4,NaN,NaN,NaN,0,NaN,NaN,,FIRMA
6,NaN,NaN,NaN,0,NaN,NaN,,FIRMA


In [40]:
required_signatures=pd.concat([required_signatures,pd.DataFrame({'firma': ' ','titulo': 'FIRMA','status':0},index=[len(required_signatures)+i])])

In [41]:
comisiones

,id,order_id,seller_id,percentage,dgi,created_at,updated_at,description,iniciales,seller_name
0,2196,800,4,0.0001,0,2025-07-10,2025-07-10,DGI,HCCL,HERREÑO\tCORRALES\tCHRISTIAN LEONARDO
1,2197,800,5,0.0001,0,2025-07-10,2025-07-10,DGI,HCFA,HERREÑO\tCORRALES\tFREDY ALEXANDER
2,2198,800,28,0.0001,0,2025-07-10,2025-07-10,DGI,HMNE,HERREÑO\tMORENO\tNELSON ERNESTO


In [32]:
payload['completer']

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [2]:
500*24

12000

In [97]:
payload['order']

{'id': 808,
 'invoice': '3469',
 'date': datetime.date(2025, 7, 31),
 'customer_id': 782,
 'seller_id': 2,
 'date_delivery': datetime.date(2025, 8, 26),
 'instalation_date': datetime.date(2025, 8, 26),
 'shipment': 'Sí',
 'customer_shipping_address_id': 469,
 'coin_id': 1,
 'subtotal': 46000.0,
 'iva': 7360.0,
 'total': 53360.0,
 'payment_conditions': '1',
 'pays': None,
 'observations': 'SE HACE ENTREGA EN  VILLAHERMOSA.',
 'status': 'AUTORIZADO',
 'authorization_id': 1,
 'created_at': numpy.datetime64('2025-07-31T08:48:25.000000000'),
 'updated_at': numpy.datetime64('2025-07-31T10:41:09.000000000'),
 'comision': 0.03,
 'reg_date': datetime.date(2025, 7, 31),
 'oc': '0',
 'ncontrato': '0',
 'isr': 0.0,
 'ieps': 0.0,
 'descuento': 0.0,
 'ncotizacion': '0',
 'otra': 0.0,
 'dgi': 0.0,
 'noha': 130,
 'ret': None,
 'tasa': 0.0,
 'description': 'SD',
 'category': 'Servicios',
 'kilos': 0.0,
 'medio_id': 4,
 'marca': 9,
 'payment_observations': 'FACTURAR: EL 100 % ANTICIPO.',
 'vent_auth': 1

11.0

In [83]:
payload['customer']['customer_street']

'Av. Coyoacan'

In [45]:
required_signatures

,id,order_id,auth_id,status,created_at,updated_at,firma,titulo
0,1982,808,2,1,2025-07-31,2025-07-31,ADGom,GERENTE DE VENTAS
1,1983,808,3,1,2025-07-31,2025-07-31,MR,GERENTE ADMINISTRATIVO


In [39]:
payload['items']

[{'id': 809,
  'internal_order_id': 808,
  'item': '1',
  'amount': 1,
  'unit': 'SERVICIO',
  'family': 'RACKS',
  'code': None,
  'description': 'SERVICIO DE TRASLADO DE XHAHUENTO A CEDIS EN VILLAHERMOSA.',
  'unit_price': 46000.0,
  'import': 46000.0,
  'created_at': numpy.datetime64('2025-07-31T08:48:25.000000000'),
  'updated_at': numpy.datetime64('2025-07-31T08:48:25.000000000'),
  'fab': 'Servicios',
  'sku': 'VT',
  'racks': None,
  'subfamimlia': None,
  'categoria': None,
  'products': '1',
  'subfamilia': None}]

In [8]:
hpagos['date']

0    2025-04-02
1    2025-05-13
2    2025-05-13
Name: date, dtype: object

In [4]:
858748.0*2

1717496.0

In [27]:
facturas['estado'].unique()

array(['autorizado', 'CANCELADO'], dtype=object)

In [19]:
print(pd.read_sql(f'select factures.*,cobro_factures.cobro_id cobro_id from factures inner join cobro_factures on cobro_factures.facture_id=factures.id where order_id ={id}',cnx))

    id  order_id  ordinal facture    amount   status npagos  created_at  \
0  881       687        1  A 4176  323327.0  CAPTURA   None  2025-03-06   
1  881       687        1  A 4176  323327.0  CAPTURA   None  2025-03-06   
2  881       687        1  A 4176  323327.0  CAPTURA   None  2025-03-06   
3  881       687        1  A 4176  323327.0  CAPTURA   None  2025-03-06   

   updated_at        date  coin_id   tc  cobro_id  
0  2025-03-06  2025-02-28        1  1.0      1039  
1  2025-03-06  2025-02-28        1  1.0      1078  
2  2025-03-06  2025-02-28        1  1.0      1103  
3  2025-03-06  2025-02-28        1  1.0      1138  


c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [7]:
400/28

14.285714285714286

In [86]:
!pip install num2words


     -------------------------------------- 163.5/163.5 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=afe6adc34437dd39b61d7cad7e4f209ed1c7f2e0c0eb60fed4a389409ed9654a
  Stored in directory: c:\users\fgonzalez\appdata\local\pip\cache\wheels\7c\d7\8d\2156234738063e3d4a39ba77dc677046100e62766b53807189
Successfully built docopt



[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
cobros=pd.read_sql("""Select cobros.* ,
    customers.customer,customers.customer_suburb, customers.clave,
    internal_orders.invoice, internal_orders.payment_conditions,
    internal_orders.category,internal_orders.description,internal_orders.status,
    coins.exchange_sell, coins.coin, coins.symbol,
    banks.bank_description,
    capturistas.name as capturista, revisores.name as revisor, autorizadores.name as autorizador
    from (((((((
    cobros inner join internal_orders on internal_orders.id = cobros.order_id) 
    inner join customers on customers.id = internal_orders.customer_id )
    inner join coins on internal_orders.coin_id = coins.id)
    left join users as capturistas on cobros.capturo=capturistas.id)
    left join users as revisores on cobros.reviso=revisores.id)
    left join users as autorizadores on cobros.autorizo=autorizadores.id))
    inner join banks on banks.id=cobros.bank_id """,cnx)

In [9]:
cobros.loc[cobros['order_id']==681]

NameError: name 'cobros' is not defined

In [7]:
459692.30/2

229846.15

In [20]:
orders.loc[orders['date'].astype(str)>str(date.strftime("%Y"))+'-01-01',"total"]

423    5.183228e+04
424    1.003168e+05
425    1.026716e+04
426    1.295720e+04
427    1.088080e+05
428    1.128912e+05
429    6.275600e+04
430    7.946000e+04
431    6.739600e+04
432    4.609944e+03
433    8.120000e+03
434    3.961400e+05
435    5.707200e+04
436    2.587960e+05
437    8.651280e+03
438    1.063720e+05
439    2.489360e+03
440    1.651724e+06
441    2.683672e+05
442    3.209720e+05
443    3.209720e+05
444    3.430120e+05
445    1.894280e+05
446    6.275832e+04
447    0.000000e+00
448    1.940100e+04
449    4.369720e+04
450    1.263240e+05
451    0.000000e+00
452    2.487040e+05
453    9.326400e+04
454    2.379160e+06
455    0.000000e+00
456    4.717720e+06
457    2.379160e+06
458    4.609944e+03
459    6.496000e+04
460    1.476680e+05
461    0.000000e+00
462    3.781600e+04
463    0.000000e+00
464    0.000000e+00
465    5.707200e+04
466    0.000000e+00
Name: total, dtype: float64

In [16]:
orders.loc[orders['date'].astype(str)>'2025-01-01','total'].sum()

14925724.8488

In [19]:
str(date.strftime("%Y"))+'01-01'

'202501-01'

In [ ]:
import datetime
from dateutil.relativedelta import relativedelta
import numpy as np

year = datetime.date.today().year
quincena=12
month = np.ceil(quincena/ 2)
isFirstHalf = quincena % 2 != 0
startDate =  str(year)+"-"+str(int(month)).zfill(2)+"-01" if isFirstHalf else  str(year)+"-"+str(int(month)).zfill(2)+"-15"
endDate =  str(year)+"-"+str(int(month)).zfill(2)+"-14" if isFirstHalf else  str((datetime.datetime(year,int(month),1 )+relativedelta(months=1))-datetime.timedelta(days=1))[:10];

2025-06-15 2025-06-30


In [ ]:
#siguiente mes en dia primero


'2025-05-31'

In [11]:
'1'.zfill(2)

'01'

In [19]:
facturas.loc[facturas['id'].isin(cobros_facturas.loc[cobros_facturas['cobro_id'].isin(cobros['cobro_id'].values)].facture_id.values)]

,id,order_id,ordinal,facture,amount,status,npagos,created_at,updated_at,date,coin_id,tc
1,826,620,1,A 4115,818738.0,CAPTURA,None,2025-01-10,2025-01-10,2025-01-10,1,1.0
2,835,620,1,A 4125,1309980.0,CAPTURA,None,2025-01-27,2025-01-27,2025-01-21,1,1.0


In [20]:
facturas

,id,order_id,ordinal,facture,amount,status,npagos,created_at,updated_at,date,coin_id,tc
0,821,620,1,A 4110,818738.0,CAPTURA,None,2025-01-03,2025-01-03,2024-12-30,1,1.0
1,826,620,1,A 4115,818738.0,CAPTURA,None,2025-01-10,2025-01-10,2025-01-10,1,1.0
2,835,620,1,A 4125,1309980.0,CAPTURA,None,2025-01-27,2025-01-27,2025-01-21,1,1.0


In [23]:
818738.0+1309980.0

2128718.0

In [42]:
for i in cobros.cobro_id.unique():
    print(i)
    fac_ids=cobros_facturas.loc[cobros_facturas['cobro_id']==i].facture_id.unique()
    for j in fac_ids:
        print('',j)
        print(len(facturas.loc[facturas['id']==j,['id','facture']]))

214
 150
1
 159
1
215
 151
1
216
 152
1
 153
1
217
 154
1
 155
1
249
 157
1
 158
1
 212
1
483
 417
1
 426
0
 440
1
 441
1
 442
0
487
 374
1
 443
1
542
 455
1
573
 515
1
635
 515
1
665
 544
1


In [10]:
cobros.facture_id

AttributeError: 'DataFrame' object has no attribute 'facture_id'

In [32]:
all_facturas

NameError: name 'all_facturas' is not defined

In [18]:
cobros_facturas.loc[cobros_facturas['cobro_id']==679]

,cobro_id,facture_id,id,created_at,updated_at
639,679,167,1034,2025-01-26,2025-01-26


In [26]:
allcobros['ordinal']=range(1,len(allcobros)+1)

C:\Users\hp\AppData\Local\Temp\ipykernel_14000\141872390.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  allcobros['ordinal']=range(1,len(allcobros)+1)


In [27]:
allcobros

,cobro_id,order_id,id,created_at,updated_at,amount,comp,date,bank_id,customer_id,invoice,noha,seller_id,comision,total,ordinal
33,58,155,40,None,None,1252626.0,1862,2022-12-29,2,446,2950,2022,14,0.003,2505252.0,1
34,59,155,41,None,None,1002100.8,1940,2023-03-28,2,446,2950,2022,14,0.003,2505252.0,2
540,595,155,580,2024-03-01,2024-03-01,250525.2,2297,2024-03-01,2,446,2950,2022,14,0.003,2505252.0,3


In [ ]:
cobros[['invoive']]

,cobro_id,order_id,id,created_at,updated_at,amount,comp,date,bank_id,customer_id,invoice,noha,seller_id,comision,total
0,21,116,4,None,None,150000.00,1066,2020-02-24,2,506,2175,12020,2,0.030,356120.00
1,22,116,5,None,None,20000.00,1472,2021-08-16,2,506,2175,12020,2,0.030,356120.00
2,23,116,6,None,None,20000.00,1492,2021-09-17,2,506,2175,12020,2,0.030,356120.00
3,24,116,7,None,None,23200.00,1526,2021-11-12,2,506,2175,12020,2,0.030,356120.00
4,25,116,8,None,None,23200.00,1552,2021-12-23,2,506,2175,12020,2,0.030,356120.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
632,695,332,688,2024-05-17,2024-05-17,55482.68,2381,2024-05-16,1,418,3102,1412023,14,0.003,1090632.00
633,695,451,689,2024-05-17,2024-05-17,92624.84,2381,2024-05-16,1,418,3102,1412023,14,0.003,1090632.00
634,696,438,690,2024-05-17,2024-05-17,444048.00,2382,2024-05-16,2,337,3191,42,5,0.003,1110120.00
635,697,398,691,2024-05-17,2024-05-17,502194.16,2383,2024-05-17,2,424,3160,11,2,0.003,1062388.32
